**Neural network experiment**

# **import user-specified packages and google drive files**

In [1]:
%xmode Verbose

Exception reporting mode: Verbose


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import dataset_confs
from DatasetManager import DatasetManager

In [4]:
csv_files = {
    #"bpic2011": ["BPIC11_f%s"%formula for formula in range(2,3)],
    "bpic2015": ["BPIC15_%s_f2"%(municipality) for municipality in range(5,6)],
    #"sepsis_cases": ["sepsis_cases_2", "sepsis_cases_4"],
    # "bpic2012": ["bpic2012_O_ACCEPTED-COMPLETE"],
    #"production": ["Production"],
    #"bpic2017": ["BPIC17_O_Accepted","BPIC17_O_Cancelled","BPIC17_0_Refused"],
    #"traffic_fines": ["traffic_fines_%s"%formula for formula in range(1,2)],
    #"hospital_billing": ["hospital_billing_%s"%suffix for suffix in [2,3]]
}
files = []
for k, v in csv_files.items():
    files.extend(v)
dataset_ref_to_datasets = {
    #"bpic2011": ["bpic2011_f%s"%formula for formula in range(2,3)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(5,6)],
   #"sepsis_cases": [ "sepsis_cases_2", "sepsis_cases_4"],
   #"bpic2012": ["bpic2012_accepted"],
   #"production": ["production"],
   #"bpic2017": ["bpic2017_accepted","bpic2017_cancelled","bpic2017_refused"],
   #"traffic_fines": ["traffic_fines_%s"%formula for formula in range(1,2)],
   #"hospital_billing": ["hospital_billing_%s"%suffix for suffix in [2,3]]
}

files = []
for k, v in csv_files.items():
    files.extend(v)
datasets = []
for k, v in dataset_ref_to_datasets.items():
    datasets.extend(v)
res = {datasets[i]: files[i] for i in range(len(datasets))}

In [5]:
print(datasets, res)

['bpic2015_5_f2'] {'bpic2015_5_f2': 'BPIC15_5_f2'}


# **import packages and functions**

In [6]:
#functions and packages
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.metrics import roc_auc_score
from sklearn.base import BaseEstimator, TransformerMixin
from pandas.api.types import is_string_dtype
from collections import OrderedDict
import matplotlib.pyplot as plt
import time
import random
from numpy import savetxt
plt.style.use('fivethirtyeight')

#LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Nadam, Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow.keras.utils as ku
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from scipy.spatial import distance

#https://towardsdatascience.com/using-neural-networks-with-embedding-layers-to-encode-high-cardinality-categorical-variables-c1b872033ba2
class ColumnEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.columns = None
        self.maps = dict()

    def transform(self, X):
        X_copy = X.copy()
        for col in self.columns:
            # encode value x of col via dict entry self.maps[col][x]+1 if present, otherwise 0
            X_copy.loc[:,col] = X_copy.loc[:,col].apply(lambda x: self.maps[col].get(x, -1)+1)
        return X_copy

    def inverse_transform(self, X):
        X_copy = X.copy()
        for col in self.columns:
            values = list(self.maps[col].keys())
            # find value in ordered list and map out of range values to None
            X_copy.loc[:,col] = [values[i-1] if 0<i<=len(values) else None for i in X_copy[col]]
        return X_copy

    def fit(self, X, y=None):
        # only apply to string type columns
        self.columns = [col for col in X.columns if is_string_dtype(X[col])]
        for col in self.columns:
            self.maps[col] = OrderedDict({value: num for num, value in enumerate(sorted(set(X[col])))})
        return self

def prepare_inputs(X_train, X_test, data):  
    global ce
    ce = ColumnEncoder()
    X_train, X_test = X_train.astype(str), X_test.astype(str)
    X_train_enc = ce.fit_transform(X_train)
    X_test_enc = ce.transform(X_test)
    return X_train_enc, X_test_enc

def create_index(log_df, column):
    """Creates an idx for a categorical attribute.
    Args:
        log_df: dataframe.
        column: column name.
    Returns:
        index of a categorical attribute pairs.
    """
    temp_list = temp_list = log_df[log_df[column] != 'none'][[column]].values.tolist() #remove all 'none' values from the index
    subsec_set = {(x[0]) for x in temp_list}
    subsec_set = sorted(list(subsec_set))
    alias = dict()
    if column !='next_activity':
      for i, _ in enumerate(subsec_set):          
          alias[subsec_set[i]] = i + 1
      alias['none'] = 0
    else:
      for i, _ in enumerate(subsec_set):
          alias[subsec_set[i]] = i  
    #reorder by the index value
    alias = {k: v for k, v in sorted(alias.items(), key=lambda item: item[1])}
    return alias

def create_indexes(i, data):
    dyn_index = create_index(data, i)
    index_dyn = {v: k for k, v in dyn_index.items()}
    dyn_weights = ku.to_categorical(sorted(index_dyn.keys()), len(dyn_index))
    return dyn_weights,  dyn_index, index_dyn

def normalize_events(log_df,features):
    """[summary]

    Args:
        log_df (DataFrame): The dataframe with eventlog data
        args (Dictionary): The set of parameters
        features (list): the list of feature name

    Returns:
        Dataframe: Returns a Dataframe with normalized numerical features
    """
    for feature in features:
            min_feature= np.min(log_df[feature])
            max_feature= np.max(log_df[feature])
            if max_feature ==0 or max_feature-min_feature==0:
                max_feature+=0.00001
            minmax = lambda x: ((x[feature]-min_feature)/(max_feature-min_feature))
            log_df[feature] = log_df.apply(minmax, axis=1)
    return log_df

# **Own created functions**

In [7]:
def groupby_caseID(data, cols):
    ans = [pd.DataFrame(y) for x, y in data[cols].groupby('Case ID', as_index=False)]
    return ans

def labels_after_grouping(data_train,data_test):
    train_labels = []
    for i in range (0,len(data_train)):
        temp_label = data_train[i]['label'].iloc[0]
        train_labels.append(temp_label)

    test_labels = []
    for i in range (0,len(data_test)):
        temp_label = data_test[i]['label'].iloc[0]
        test_labels.append(temp_label)
    train_y = [1 if i!='regular' else 0 for i in train_labels]
    test_y = [1 if i!='regular' else 0 for i in test_labels]
    return train_y, test_y

def pad_data(cols, data, maxlen):
    #padding of the different categorical columns
    #train paddings
    paddings = []
    for i in cols:
        padding= []
        for k in range(0,len(data)):
            temp = []
            temp = list(data[k][i])
            padding.append(temp)
        padded = np.array(pad_sequences(padding,maxlen=maxlen, padding='pre', truncating='pre',value=0))
        paddings.append(padded)
    return paddings

def reshape_num_data(pad_data, cutoff):
        pad_num = np.reshape(pad_data, (len(pad_data), cutoff, 1))
        return pad_num
    
def processString(txt):
    specialChars = "():. " 
    for specialChar in specialChars:
      txt = txt.replace(specialChar, '')
    return txt

# function for preprocessing data
def create_data_groupby_and_label(data, dt_train_prefixes, dt_test_prefixes, cols, dataset_manager, cat_cols, num_cols):

    #cat columns integerencoded
    dt_train_prefixes[cat_cols],dt_test_prefixes[cat_cols]= prepare_inputs(dt_train_prefixes[cat_cols], dt_test_prefixes[cat_cols], data)
    dt_train_prefixes[cat_cols] = dt_train_prefixes[cat_cols]+1
    dt_test_prefixes[cat_cols] = dt_test_prefixes[cat_cols]+1
   
    #[-1,1] normalize the numerical columns
    dt_train_prefixes = normalize_events(dt_train_prefixes,num_cols)
    dt_test_prefixes = normalize_events(dt_test_prefixes,num_cols)

    #groupby case ID
    ans_train = groupby_caseID(dt_train_prefixes, cols)
    ans_test = groupby_caseID(dt_test_prefixes, cols)
    
    #obtain the new label lists after grouping
    train_y, test_y = labels_after_grouping(ans_train, ans_test)

    #obtain the new number of events after grouping
    nr_events_all = list(dt_test_prefixes.groupby('Case ID', as_index=False).last()["prefix_nr"])
    
    return dt_train_prefixes, dt_test_prefixes, train_y, test_y, nr_events_all


def transform_train_data(train, cols, dynamic_cat_cols, dynamic_num_cols, static_cat_cols, static_num_cols,activity_col, maxlen):
    #groupby case ID
    ans_train = groupby_caseID(train, cols)
    
    ######ACTIVITY_COL########
    activity_train = pad_data(activity_col, ans_train, maxlen)

    ######DYNAMIC_COLS########
    paddings_train = pad_data(dynamic_cat_cols, ans_train, maxlen)
    paddings_train2 = pad_data(dynamic_num_cols, ans_train, maxlen)

    #STATIC COLUMNS
    #static_cat_cols
    pad_train =pad_data(static_cat_cols, ans_train, maxlen)

    #static_num_cols 
    pad_train2 =pad_data(static_num_cols, ans_train, maxlen)
   
    return paddings_train, paddings_train2, pad_train, pad_train2, activity_train
  
def transform_test_data(dt_test_prefixes, cols, dynamic_cat_cols, dynamic_num_cols, static_cat_cols, static_num_cols, activity_col, maxlen):
    
    ans_test = groupby_caseID(dt_test_prefixes,cols)
    ######ACTIVITY_COL########
    activity_test = pad_data(activity_col, ans_test, maxlen)
    ####DYNAMIC COLS####
    paddings_test = pad_data(dynamic_cat_cols, ans_test, maxlen)
    paddings_test2 = pad_data(dynamic_num_cols, ans_test, maxlen)

    pad_test = pad_data(static_cat_cols, ans_test, maxlen)
    pad_test2 = pad_data(static_num_cols, ans_test, maxlen)
  
    return paddings_test, paddings_test2, pad_test, pad_test2, activity_test

def create_inputs_test(data,paddings_test, paddings_test2, pad_test, pad_test2, activity_test, activity_col, dynamic_cat_cols, dynamic_num_cols, static_cat_cols, static_num_cols, maxlen):
    
    model_inputs_test = []
    for i in range(0,len(activity_col)):
        dyn_weights, dyn_index, index_dyn = create_indexes(activity_col[i], data)
        input_ohe_test = to_categorical(activity_test[i], num_classes=len(dyn_index)+1)
        input_ohe_test = input_ohe_test[:, :, 1:]
        model_inputs_test.append(input_ohe_test) 
        
    for i in range(0,len(dynamic_cat_cols)):
        dyn_weights, dyn_index, index_dyn = create_indexes(dynamic_cat_cols[i], data)
        input_ohe_test = to_categorical(paddings_test[i], num_classes=len(dyn_index)+1)
        input_ohe_test = input_ohe_test[:, :, 1:]
        model_inputs_test.append(input_ohe_test)  
        
    for i in range(0,len(dynamic_num_cols)):
        model_inputs_test.append(paddings_test2[i])

    for i in range(0,len(static_cat_cols)):
        stat_weights, stat_index, index_stat = create_indexes(static_cat_cols[i], data)
        input_ohe_test = to_categorical(pad_test[i], num_classes=len(stat_index)+1)
        input_ohe_test = input_ohe_test[:, :, 1:]
        model_inputs_test.append(input_ohe_test)

    for i in range(0,len(static_num_cols)):
        model_inputs_test.append(reshape_num_data(pad_test2[i], maxlen))

    return model_inputs_test

def attack_event_payload(dt_train, dynamic_attributes):
    print('first attack generated')
    dt_train2 =dt_train.copy()
    tic = time.perf_counter()
    tic = time.perf_counter()
    dt_last = dt_train2.groupby(['Case ID']).last()
    dt_last = dt_last.reset_index()
    for i in range(0, max_prefix_length):
        for j in dynamic_attributes:
            dt_last[j].loc[(dt_last.prefix_nr == i+1)] = random.choices(
                payload_values[j], k=len(dt_last.loc[(dt_last.prefix_nr == i+1)]))
    dt_train = dt_train.set_index('Case ID','prefix_nr','event_nr')
    dt_last = dt_last.set_index('Case ID','prefix_nr','event_nr')
    dt_train.update(dt_last)
    dt_train = dt_train.reset_index()
    dt_train = dt_train.drop('level_0', axis=1)
    toc = time.perf_counter()

    print(f"First adversarial attack in {toc - tic:0.4f} seconds")
    return dt_train

def attack_all_payload(dt_train, dynamic_attributes):
    print('second attack generated')
    tic = time.perf_counter()
    for j in dynamic_attributes:
        dt_train[j] = random.choices(payload_values[j], k=len(dt_train))
    toc = time.perf_counter()
    print(f"Second adversarial attack in {toc - tic:0.4f} seconds")
    return dt_train

def calculate_distance(preds_all, preds_all2, coef_a, coef_b):
    print('attack')
    array_of_distances = np.zeros((max_prefix_length, 5))
    array_of_distances[:,2] = [x+1 for x in list(prefix_lengths.values())]
    tic = time.perf_counter()
    if cls_method =='LSTM':
        explanations1 = coef_a
        explanations2 = coef_b
    row_sums = explanations1.sum(axis=1)
    norm_explanation1 = explanations1 / row_sums[:, np.newaxis]
    row_sums = explanations2.sum(axis=1)
    norm_explanation2 = explanations2 / row_sums[:, np.newaxis]
        
    for i in range(0, len(nr_events_all)):
        nr_event = nr_events_all[i]-1
        dist_temp = distance.euclidean(norm_explanation1[i],norm_explanation2[i])
        if round(preds_all[i]) == round(preds_all2[i]):
            array_of_distances[nr_event, 0] += dist_temp
        else:
            array_of_distances[nr_event, 1] += dist_temp
            if round(preds_all2[i]) != test_y_all[i]:
               array_of_distances[nr_event, 4] += 1
            else:
               array_of_distances[nr_event, 3] += 1
    toc = time.perf_counter()
    print(f"adversarial attack in {toc - tic:0.4f} seconds")
    return array_of_distances

# **LSTM Model**

In [8]:
def LSTM_model(paddings_train, paddings_train2, pad_train, pad_train2, activity_train):
    feature_names_all = []
    LSTMlayers = []
    input_layers = []
    alpha = layers.Bidirectional(layers.LSTM(args['lstm_size_alpha'], dropout=0.3, return_sequences=True),
                                                name='alpha')
    alpha_dense = layers.Dense(1, kernel_regularizer=l2(0.001))
    model_inputs = []
    #model_inputs_test = []
    for i in range(0,len(activity_col)):
        dyn_weights, dyn_index, index_dyn = create_indexes(activity_col[i], data)
        for key in range(len(index_dyn)):
                feature_names_all.append(str(index_dyn[key]))
        input_ohe = to_categorical(activity_train[i], num_classes=len(dyn_index)+1)
        #remove mask
        input_ohe = input_ohe[:, :, 1:]
        model_inputs.append(input_ohe)
        dyn_cat_name =processString(activity_col[i])
        input_layer = layers.Input(shape=(input_ohe.shape[1],len(dyn_index), ), name=dyn_cat_name)
        input_layers.append(input_layer)
        #Set up the LSTM networks
        beta = layers.Bidirectional(layers.LSTM(args['lstm_size_beta'], dropout=0.3, return_sequences=True), name='beta'+str(dyn_cat_name))
        beta_out = beta(input_layer)
        #Dense layer for attention
        betadense = layers.Dense(dyn_weights.shape[1],activation='tanh', kernel_regularizer=l2(0.001))
        beta_out = layers.TimeDistributed(betadense, name='feature_attention_'+dyn_cat_name)(beta_out)
        c_t = layers.Multiply(name = dyn_cat_name+'_importance')([beta_out,input_layer])
        LSTMlayers.append(c_t)
        
    for i in range(0,len(dynamic_cat_cols)):
        dyn_weights,  dyn_index, index_dyn = create_indexes(dynamic_cat_cols[i], data)
        for key in range(len(index_dyn)):
                feature_names_all.append(str(index_dyn[key]))
        input_ohe = to_categorical(paddings_train[i], num_classes= len(dyn_index)+1)
        #remove mask
        input_ohe = input_ohe[:, :, 1:]
        model_inputs.append(input_ohe)
        dyn_cat_name =processString(dynamic_cat_cols[i])
        input_layer = layers.Input(shape=(input_ohe.shape[1],len(dyn_index), ), name=dyn_cat_name)
        input_layers.append(input_layer)
        #Set up the LSTM networks
        beta = layers.Bidirectional(layers.LSTM(args['lstm_size_beta'], dropout=0.3, return_sequences=True),
                                             name='beta'+str(dyn_cat_name))
        beta_out = beta(input_layer)
        #Dense layer for attention
        betadense = layers.Dense(dyn_weights.shape[1],
                                         activation='tanh', kernel_regularizer=l2(0.001))

        beta_out = layers.TimeDistributed(betadense, name='feature_attention_'+dyn_cat_name)(beta_out)
        c_t = layers.Multiply(name = dyn_cat_name+'_importance')([beta_out,input_layer])
        LSTMlayers.append(c_t)

    for i in range(0,len(dynamic_num_cols)):
        dyn_num_name =processString(dynamic_num_cols[i])
        input_layer = layers.Input(shape=(cutoff,1, ), name=dyn_num_name)
        feature_names_all.append(dynamic_num_cols[i])
        model_inputs.append(paddings_train2[i])
        input_layers.append(input_layer)
        #Set up the LSTM networks
        beta = layers.Bidirectional(layers.LSTM(args['lstm_size_beta'], dropout=0.3, return_sequences=True),
                                             name='beta'+str(dyn_num_name))
        beta_out = beta(input_layer)
        #Dense layer for attention
        betadense = layers.Dense(1,activation='tanh', kernel_regularizer=l2(0.001))

        beta_out = layers.TimeDistributed(betadense, name='feature_attention_'+dyn_num_name)(beta_out)
        c_t = layers.Multiply(name = dyn_num_name+'_importance')([beta_out,input_layer])
        LSTMlayers.append(c_t)

    for i in range(0,len(static_cat_cols)):
        stat_weights, stat_index, index_stat = create_indexes(static_cat_cols[i], data)
        for key in range(len(index_stat)):
                feature_names_all.append(str(index_stat[key]))
        input_ohe = to_categorical(pad_train[i], num_classes=len(stat_index)+1)
        input_ohe = input_ohe[:, :, 1:]
        model_inputs.append(input_ohe)
        stat_cat_name =processString(static_cat_cols[i])
        input_layer = layers.Input(shape=(input_ohe.shape[1],len(stat_index), ), name=stat_cat_name)
        input_layers.append(input_layer)
        betadense = layers.Dense(1, activation='tanh', kernel_regularizer=l2(0.001))
        beta_out = layers.TimeDistributed(betadense, name='feature_attention_'+stat_cat_name)(input_layer)
        c_t = layers.Multiply(name = stat_cat_name+'_importance')([beta_out,input_layer])
        LSTMlayers.append(c_t)
    
    for i in range(0,len(static_num_cols)):
        static_num_name =processString(static_num_cols[i])
        input_layer = layers.Input(shape=(cutoff,1), name=static_num_name)
        input_layers.append(input_layer)
        feature_names_all.append(static_num_cols[i])
        betadense = layers.Dense(1, activation='tanh', kernel_regularizer=l2(0.001))
        beta_out = layers.TimeDistributed(betadense, name='feature_attention_'+static_num_name)(input_layer)
        c_t = layers.Multiply(name = static_num_name+'_importance')([beta_out,input_layer])
        LSTMlayers.append(c_t)
        model_inputs.append(reshape_num_data(pad_train2[i], cutoff))
  
    total_layers = LSTMlayers
    c_t = layers.concatenate(total_layers,name = 'concat')
    #Compute alpha, timestep attention
    alpha_out = alpha(c_t)
    alpha_out = layers.TimeDistributed(alpha_dense, name='alpha_dense')(alpha_out)
    alpha_out = layers.Softmax(name='timestep_attention', axis=1)(alpha_out)

    #Compute context vector based on attentions and embeddings
    c_t = layers.Multiply()([alpha_out, c_t])
    c_t = layers.Lambda(lambda x: backend.sum(x, axis=1))(c_t)

    #contexts = L.concatenate([c_t,age_input,cl_input_d], name='contexts')
    contexts = layers.Dropout(0.3)(c_t)

    output_layer = Dense(1, activation='sigmoid', name='final_output')(contexts)
        
    #MODEL
    model = Model(inputs=[input_layers], outputs=output_layer)

    model.compile(loss={'final_output':'binary_crossentropy'}, optimizer= Nadam(learning_rate=0.0001))

    model.summary()
    return model_inputs, model, feature_names_all

# **Attention**

In [9]:
def create_context(output_with_attention, timestep_att, counter,m):
    contexts = output_with_attention[counter][m]
    contexts = contexts.T*timestep_att
    contexts = contexts.T
    contexts = contexts.sum(axis=0).tolist()
    return contexts

def attention_values(model_attn_weights,model_inputs_test, n):
    att_values_total = np.zeros((len(model_inputs_test[0]), n))  
    output_with_attention = model_attn_weights.predict(model_inputs_test)[0]
    for m in range(0,len(model_inputs_test[0])):
        counter = 2
        att_values = []
        timestep_att = output_with_attention[1][m].flatten()
        for i in range(0,len(activity_col)):
            contexts = create_context(output_with_attention, timestep_att, counter, m)
            att_values.extend(contexts)
            counter +=1
        for i in range(0,len(dynamic_cat_cols)):
            contexts = create_context(output_with_attention, timestep_att, counter, m)
            att_values.extend(contexts)
            counter +=1
        for i in range(0,len(dynamic_num_cols)):
            contexts = create_context(output_with_attention, timestep_att, counter, m)
            att_values.extend(contexts)
            counter +=1
        for i in range(0,len(static_cat_cols)):
            contexts = create_context(output_with_attention, timestep_att, counter, m)
            att_values.extend(contexts)
            counter +=1
        for i in range(0,len(static_num_cols)):
            contexts = create_context(output_with_attention, timestep_att, counter, m)
            att_values.extend(contexts)
            counter +=1
        att_values_total[m,:] = att_values
    return att_values_total

# **parameters**

In [10]:
# parameters
#terminology 
params_dir = './params_dir_DL'
results_dir = '/content/drive/MyDrive/CurrentWork/Robustness/results_dir_DL_test'
column_selection= 'all'
cls_encoding ='OHE'
classifiers =['LSTM']
n_iter = 1
n_splits = 3
train_ratio = 0.8
random_state = 22
l2reg = 0.001

allow_negative=False
incl_time = True 
incl_res = True
# create results directory
if not os.path.exists(os.path.join(results_dir)):
    os.makedirs(os.path.join(results_dir))

# **loop over datasets and classifiers**

In [13]:
for cls_method in classifiers:
  for dataset_name in datasets: 
    print('Dataset:', dataset_name)
    print('Classifier', cls_method)
    print('Encoding', cls_encoding)
    method_name = "%s_%s"%(column_selection, cls_encoding)            

    optimal_params_filename = os.path.join("", "optimal_params_%s_%s_%s.pickle" % (cls_method, dataset_name, method_name))
    if not os.path.isfile(optimal_params_filename) or os.path.getsize(optimal_params_filename) <= 0:
      print('problem')
    with open(optimal_params_filename, "rb") as fin:
      args = pickle.load(fin)
      print(args)
                
    # read the data
    dataset_manager = DatasetManager(dataset_name)
    data = dataset_manager.read_dataset()  
    #dataset_name_csv = res[dataset_name]
    #data = pd.read_csv(dataset_name_csv+'.csv', sep=';')
        
    cls_encoder_args = {'case_id_col': dataset_manager.case_id_col, 
                            'static_cat_cols': dataset_manager.static_cat_cols,
                            'static_num_cols': dataset_manager.static_num_cols, 
                            'dynamic_cat_cols': dataset_manager.dynamic_cat_cols,
                            'dynamic_num_cols': dataset_manager.dynamic_num_cols, 
                            'fillna': True}
    #file to save results
    outfile = os.path.join(results_dir, "performance_results_%s_%s_%s.csv" % (cls_method, dataset_name, method_name))
    outfile1 = os.path.join(results_dir, "attack1_%s_%s_%s.csv" % (cls_method, dataset_name, method_name))   
    outfile2 = os.path.join(results_dir, "attack2_%s_%s_%s.csv" % (cls_method, dataset_name, method_name)) 

    # determine min and max (truncated) prefix lengths
    min_prefix_length = 1
    if "traffic_fines" in dataset_name:
      max_prefix_length = 10
    elif "bpic2017" in dataset_name:
      max_prefix_length = min(20, dataset_manager.get_pos_case_length_quantile(data, 0.90))
    else:
      max_prefix_length = min(40, dataset_manager.get_pos_case_length_quantile(data, 0.90))
    maxlen = cutoff = max_prefix_length
            
    # split into training and test
    train, test = dataset_manager.split_data_strict(data, train_ratio, split="temporal")
        
    preds_all = []
    preds_all_attack1 = []
    preds_all_attack2 = []
    test_y_all = []
    test_y_all_attack1 = []
    test_y_all_attack2 = []    
            
    #prefix generation of train and test data
    dt_train_prefixes = dataset_manager.generate_prefix_data(train, min_prefix_length, max_prefix_length)
    dt_test_prefixes = dataset_manager.generate_prefix_data(test, min_prefix_length, max_prefix_length)

    dynamic_cat_cols = cls_encoder_args['dynamic_cat_cols'].copy()
    #ACTIVITY COL
    activity_col =  [x for x in dynamic_cat_cols if 'Activity' in x]
    dynamic_cat_cols.remove(activity_col[0])
    dynamic_num_cols = cls_encoder_args['dynamic_num_cols'].copy()
    dynamic_num_cols.remove('event_nr')
    static_cat_cols = cls_encoder_args['static_cat_cols'].copy()
    static_num_cols = cls_encoder_args['static_num_cols'].copy()
    #COLUMNS
    cat_cols = activity_col + dynamic_cat_cols + static_cat_cols
    num_cols = dynamic_num_cols + static_num_cols
    dynamic_cols = dynamic_cat_cols+ dynamic_num_cols + activity_col
    static_cols = static_cat_cols+ static_num_cols
        
    #TOTAL COLS
    cols =  activity_col + dynamic_cat_cols + dynamic_num_cols + static_cat_cols + static_num_cols
        
    #groupby data per Case ID and extract label
    label_case_cols = cols + [cls_encoder_args['case_id_col']] + ['label']
       
    dynamic_attributes = dynamic_cols.copy()

    payload_values = {key: list(dt_train_prefixes[key].unique()) for key in dynamic_attributes}
        
    prefix_lengths = {key+1 : len(dt_test_prefixes[dt_test_prefixes['prefix_nr']==key+1]) for key in range(0,max_prefix_length)}
        
    #attack 1
    dt_test_prefixes_attack1 = dt_test_prefixes.copy()
    dt_train_prefixes_attack1 = dt_train_prefixes.copy()
    dt_test_prefixes_attack1 = attack_event_payload(dt_test_prefixes_attack1, dynamic_attributes)
    
    #attack 2
    dt_test_prefixes_attack2 = dt_test_prefixes.copy()
    dt_train_prefixes_attack2 = dt_train_prefixes.copy()
    dt_test_prefixes_attack2 = attack_all_payload(dt_test_prefixes_attack2, dynamic_attributes)
    
    dt_train, dt_test, train_y, test_y, nr_events_all = create_data_groupby_and_label(data, dt_train_prefixes, dt_test_prefixes, label_case_cols, dataset_manager, cat_cols, num_cols)
    test_y_all.extend(test_y)
    paddings_train, paddings_train2, pad_train, pad_train2, activity_train = transform_train_data(dt_train,label_case_cols, dynamic_cat_cols, dynamic_num_cols, static_cat_cols, static_num_cols, activity_col, maxlen)
    paddings_test, paddings_test2, pad_test, pad_test2, activity_test = transform_test_data(dt_test, label_case_cols, dynamic_cat_cols, dynamic_num_cols, static_cat_cols, static_num_cols, activity_col, maxlen)
    model_inputs, model, feature_names_all = LSTM_model(paddings_train, paddings_train2, pad_train, pad_train2, activity_train)
    model_inputs_test = create_inputs_test(data,paddings_test, paddings_test2, pad_test, pad_test2, activity_test, activity_col, dynamic_cat_cols, dynamic_num_cols, static_cat_cols, static_num_cols, maxlen)
    early_stopping = EarlyStopping(monitor='val_loss', mode='auto', patience=5,min_delta=0.001)
    lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, verbose=2, mode='auto')
    print('model fitting')
    model.fit(model_inputs,
                  np.array(train_y),
                  callbacks=[early_stopping, lr_reducer],
                  validation_split = 0.1,
                  verbose=2, batch_size=args['batch_size'],
                  epochs=100)

    pred = model.predict(model_inputs_test)
    pred3  =[pred[i][0] for i in range(0,len(pred))]
    preds_all.extend(pred3)
        
    #attack1
    dt_train_attack1, dt_test_attack1, train_y_attack1, test_y_attack1,nr_events_all_attack1 = create_data_groupby_and_label(data,dt_train_prefixes_attack1, dt_test_prefixes_attack1, label_case_cols,dataset_manager, cat_cols, num_cols)
    test_y_all_attack1.extend(test_y_attack1)
    paddings_train_attack1, paddings_train2_attack1,pad_train_attack1, pad_train2_attack1, activity_train_attack1 = transform_train_data(dt_train_attack1,label_case_cols, dynamic_cat_cols, dynamic_num_cols, static_cat_cols, static_num_cols, activity_col, maxlen)
    paddings_test_attack1, paddings_test2_attack1,pad_test_attack1, pad_test2_attack1,  activity_test_attack1 = transform_test_data(dt_test_attack1, label_case_cols, dynamic_cat_cols, dynamic_num_cols, static_cat_cols, static_num_cols, activity_col, maxlen)
    model_inputs_attack1, model_attack1, feature_names_all_attack1 = LSTM_model(paddings_train_attack1, paddings_train2_attack1, pad_train_attack1, pad_train2_attack1, activity_train_attack1)
    model_inputs_test_attack1 = create_inputs_test(data, paddings_test_attack1, paddings_test2_attack1,pad_test_attack1, pad_test2_attack1, activity_test_attack1, activity_col, dynamic_cat_cols, dynamic_num_cols, static_cat_cols, static_num_cols, maxlen)

    model_attack1.fit(model_inputs_attack1,
                  np.array(train_y),
                  callbacks=[early_stopping, lr_reducer],
                  validation_split = 0.1,
                  verbose=2, batch_size=args['batch_size'],
                  epochs=100)

    pred_attack1 = model_attack1.predict(model_inputs_test_attack1)
    pred3_attack1  =[pred_attack1[i][0] for i in range(0,len(pred_attack1))]
    preds_all_attack1.extend(pred3_attack1)

    # attack2
    dt_train_attack2, dt_test_attack2, train_y_attack2, test_y_attack2, nr_events_all_attack2 = create_data_groupby_and_label(data,dt_train_prefixes_attack2, dt_test_prefixes_attack2, label_case_cols, dataset_manager, cat_cols, num_cols)
    test_y_all_attack2.extend(test_y_attack2)
    paddings_train_attack2, paddings_train2_attack2,pad_train_attack2, pad_train2_attack2, activity_train_attack2 = transform_train_data(dt_train_attack2,label_case_cols, dynamic_cat_cols, dynamic_num_cols, static_cat_cols, static_num_cols, activity_col, maxlen)
    paddings_test_attack2, paddings_test2_attack2,pad_test_attack2, pad_test2_attack2, activity_test_attack2 = transform_test_data(dt_test_attack2, label_case_cols, dynamic_cat_cols, dynamic_num_cols, static_cat_cols, static_num_cols, activity_col, maxlen)
    model_inputs_attack2, model_attack2, feature_names_all_attack2 = LSTM_model(paddings_train_attack2, paddings_train2_attack2, pad_train_attack2, pad_train2_attack2, activity_train_attack2)
    model_inputs_test_attack2 = create_inputs_test(data,paddings_test_attack2, paddings_test2_attack2,pad_test_attack2, pad_test2_attack2,  activity_test_attack2, activity_col, dynamic_cat_cols, dynamic_num_cols, static_cat_cols, static_num_cols, maxlen)
  
    model_attack2.fit(model_inputs_attack2,
                  np.array(train_y),
                  callbacks=[early_stopping, lr_reducer],
                  validation_split = 0.1,
                  verbose=2, batch_size=args['batch_size'],
                  epochs=100)

    pred_attack2 = model_attack2.predict(model_inputs_test_attack2)
    pred3_attack2  =[pred_attack2[i][0] for i in range(0,len(pred_attack2))]
    preds_all_attack2.extend(pred3_attack2)
    print('attention')       
    #attention
    layer_names = [layer.name for layer in model.layers]
    print(layer_names)
    model_outputs = []
    model_outputs.append(model.output)
    model_outputs.append(model.get_layer('timestep_attention').output)
    all_cols = activity_col + dynamic_cat_cols + dynamic_num_cols +static_cat_cols +static_num_cols
    for i in all_cols:
            i = processString(i)
            model_outputs.append(model.get_layer(i+'_importance').output)
    model_attn_weights = Model(inputs=model.input,
                      outputs=[model_outputs])
    #attention model attack 1
    model_outputs_attack1 = []
    model_outputs_attack1.append(model_attack1.output)
    model_outputs_attack1.append(model_attack1.get_layer('timestep_attention').output)
    for i in all_cols:
            i = processString(i)
            model_outputs_attack1.append(model_attack1.get_layer(i+'_importance').output)
    model_attn_weights_attack1 = Model(inputs=model_attack1.input,
                              outputs=[model_outputs_attack1])
        
    # attention model attack 1
    model_outputs_attack2 = []
    model_outputs_attack2.append(model_attack2.output)
    model_outputs_attack2.append(model_attack2.get_layer('timestep_attention').output)
    for i in all_cols:
            i = processString(i)
            model_outputs_attack2.append(model_attack2.get_layer(i+'_importance').output)
    model_attn_weights_attack2 = Model(inputs=model_attack2.input,
                              outputs=[model_outputs_attack2])
    print('saving')     
    with open(outfile, 'w') as fout:
              fout.write("%s;%s;%s;%s;%s;%s\n"%("dataset", "method", "cls", "nr_events", "metric", "score"))
              
              dt_results = pd.DataFrame({"actual": test_y_all, "predicted": preds_all, "nr_events": nr_events_all})
              for nr_events, group in dt_results.groupby("nr_events"):
                  if len(set(group.actual)) < 2:
                      fout.write("%s;%s;%s;%s;%s;%s;%s\n"%(dataset_name, method_name, cls_method, nr_events,-1, "auc", np.nan))
                  else:
                      fout.write("%s;%s;%s;%s;%s;%s;%s\n"%(dataset_name, method_name, cls_method, nr_events,-1, "auc", roc_auc_score(group.actual, group.predicted)))
              fout.write("%s;%s;%s;%s;%s;%s\n"%(dataset_name, method_name, cls_method,-1, "auc", roc_auc_score(dt_results.actual, dt_results.predicted)))
                
              #attack1
              dt_results = pd.DataFrame({"actual": test_y_all_attack1, "predicted": preds_all_attack1, "nr_events": nr_events_all_attack1})
              for nr_events, group in dt_results.groupby("nr_events"):
                  if len(set(group.actual)) < 2:
                      fout.write("%s;%s;%s;%s;%s;%s;%s\n"%(dataset_name, method_name, cls_method, nr_events,-1, "auc", np.nan))
                  else:
                      fout.write("%s;%s;%s;%s;%s;%s;%s\n"%(dataset_name, method_name, cls_method, nr_events,-1, "auc", roc_auc_score(group.actual, group.predicted)))
              fout.write("%s;%s;%s;%s;%s;%s\n"%(dataset_name, method_name, cls_method,-1, "auc", roc_auc_score(dt_results.actual, dt_results.predicted)))

              #attack2
              dt_results = pd.DataFrame({"actual": test_y_all_attack2, "predicted": preds_all_attack2, "nr_events": nr_events_all_attack2})
              for nr_events, group in dt_results.groupby("nr_events"):
                  if len(set(group.actual)) < 2:
                      fout.write("%s;%s;%s;%s;%s;%s;%s\n"%(dataset_name, method_name, cls_method, nr_events,-1, "auc", np.nan))
                  else:
                      fout.write("%s;%s;%s;%s;%s;%s;%s\n"%(dataset_name, method_name, cls_method, nr_events,-1, "auc", roc_auc_score(group.actual, group.predicted)))
              fout.write("%s;%s;%s;%s;%s;%s\n"%(dataset_name, method_name, cls_method,-1, "auc", roc_auc_score(dt_results.actual, dt_results.predicted)))
             

Dataset: bpic2015_5_f2
Classifier LSTM
Encoding OHE
{'batch_size': 48, 'lstm_size_alpha': 8, 'lstm_size_beta': 8}
first attack generated


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:142: FutureWarning: In a future version of pandas all arguments of DataFrame.set_index except for the argument 'keys' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:143: FutureWarning: In a future version of pandas all arguments of DataFrame.set_index except for the argument 'keys' will be keyword-only


First adversarial attack in 1.5201 seconds
second attack generated
Second adversarial attack in 0.5869 seconds
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Activity (InputLayer)          [(None, 40, 377)]    0           []                               
                                                                                                  
 monitoringResource (InputLayer  [(None, 40, 16)]    0           []                               
 )                                                                                                
                                                                                                  
 question (InputLayer)          [(None, 40, 10)]     0           []                               
                                                                                